In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
# read JSON file
with open('table17aug21.json', 'r') as json_file:
    json_data = json.load(json_file)

In [3]:
# get list of buckets(lists of cards) to DataFrame
buckets = pd.DataFrame(json_data['lists']).loc[:, ['id', 'name']].set_index('id')
buckets.head()

,name
id,
5e341a4871eef21fe7cd1984,Paper accepted
5ef47d2bcc3c19064902e85d,Author submitted
5ffc260ff8ef244d7cc4e865,At team
5e341a4b43b62f44e56427e2,At editor
5e341a4f1402ce58aeabda74,At author


In [4]:
# cards to DataFrame
columns = ['id', 'idList', 'closed', 'desc', 'name', 'labels']
cards = pd.DataFrame(json_data['cards']).loc[:, columns].set_index('id')
cards_num = len(cards['labels'].values)     # get total number of cards to iterate

# extract labels from dictionary
card_labels = []
for i in range(cards_num):
    try:
        card_labels.append([cards['labels'].values[i][j]['name'] for j in range(len(cards['labels'].values[i]))])
    except IndexError:
        card_labels.append("")
        
#cards['labels'] = [ cards['labels'].values[i][0]['name'] for i in range(cards_num)]
cards['labels'] = card_labels
# add buckets (lists of cards) names to DataFrame 'cards'
cards = pd.merge(cards, buckets, how='left', left_on='idList', right_on='id', 
                 suffixes=('_card', '_list'), right_index=True).drop(['idList'], axis=1)

cards.head()

,closed,desc,name_card,labels,name_list
id,,,,,
5ed4ac37e2c0ae3fdd0564d4,False,,Template,[],Paper accepted
5ede5220637a767c05f9edcd,True,,Paper,[],Paper accepted
5ef8d7c4dba8de08b543c6bc,True,,Re: RESTUD: MS 24476-3 Accepted,[],Paper accepted
5ef8d8189e41fd6f7132b08a,True,,Re: RESTUD: MS 26198-3 Accepted,[],Paper accepted
5ef8d829b9b4db1d68a3660c,True,,Re: RESTUD: MS 26541-2 Accepted,[],Paper accepted


In [5]:
cards['labels'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[]                                                   74
[Short, Revision]                                    10
[Revision, Short]                                    10
[Stata, Revision]                                     5
[Revision]                                            4
[LARGE, Stata, Revision]                              3
[LARGE, Stata, Short, Revision]                       2
[Stata]                                               2
[Stata, Revision, Short]                              2
[Short]                                               2
[Short, Revision, LARGE]                              1
[Needs attention, LARGE, Short, Revision]             1
[Short, Revision, Needs attention]                    1
[Stata, Needs attention, Revision]                    1
[Stata, Short, Revision]                              1
[Stata, Matlab]                                       1
[LARGE, Stata, Matlab, Revision]                      1
[Mathematica, Needs attention, Revision, Short] 

In [6]:
# prepare DataFrame with actions (activities)
columns = ['idMemberCreator','data','type','date']
actions = pd.DataFrame(json_data['actions']).loc[:, columns]
print(f'actions: {set(actions["type"])}')
# filter DataFrame actions and sort in chronological order
cards_actions = ['createCard', 'updateCard', 'commentCard','addAttachmentToCard']
actions = actions[actions['type'].isin(cards_actions)].reset_index(drop=True)
actions = actions.sort_values(by='date').reset_index(drop=True)

# prepare DataFrame with memebers (users)
members = pd.DataFrame(json_data['members']).loc[:, ['id', 'username']].set_index('id')
# add usernames to DataFrame 'actions'
actions = pd.merge(actions, members, how='left',
                   left_on='idMemberCreator', right_on='id',
                   suffixes=('_actions', '_members')).drop(['idMemberCreator'], axis=1)

# add actions information to DataFrame 'actions'

# prepare new columns
actions['card_id'] = ''
actions['changed'] = ''
actions['old'] = ''
actions['new'] = ''

actions: {'updateCheckItemStateOnCard', 'addAttachmentToCard', 'deleteCard', 'removeMemberFromCard', 'updateCard', 'copyCard', 'addMemberToCard', 'createCard', 'commentCard', 'disablePlugin', 'enablePlugin'}


In [7]:
# add info to DataFrame
for row in range(len(actions)):
    actions.loc[row, 'card_id'] = actions.loc[row, 'data']['card']['id']
    try:
        actions.loc[row, 'changed'] = list(actions.loc[row, 'data']['old'].keys())[0]
    except KeyError:
        actions.loc[row, 'changed'] = ""
    
    try:
        actions.loc[row, 'old'] = list(actions.loc[row, 'data']['old'].values())
    except KeyError:
        actions.loc[row, 'old'] = ""
        # get value of what was changed to use it after
    try:    
        changed = list(actions.loc[row, 'data']['old'].keys())[0]
    except KeyError:
        changed = ""
        
    try:
        actions.loc[row, 'new'] = actions.loc[row, 'data']['card'][changed]
    except KeyError:
        actions.loc[row,'new'] = ""

In [8]:
# create table for export
cards_list = []
for row in range(len(cards)):
    # prepare empty dictionary
    card = {}
    
    # add name of card (task)
    card['name'] = cards.iloc[row,2]
    
    # add status
    card['status'] = 'Open'
    if cards.iloc[row, 0] == True:
        card['status'] = 'Closed'
    
    # add name of list (bucket)
    card['list'] = cards.iloc[row, 4]
    
    # add label
    card['label'] = cards.iloc[row, 3]
    
    # add description
    card['desc'] = cards.iloc[row, 1]
    
    # add actions to card
    # get card id
    card_id = cards.index[row]
    
    # iterate over list of action to find action for this card
    counter = 0
    for action_id in range(len(actions)):
        if actions['card_id'][action_id] == card_id:
            card['activity' + str(counter)] = actions['type'][action_id]
            card['date' + str(counter)] = actions['date'][action_id][:16]
            card['user' + str(counter)] = actions['username'][action_id]
            counter += 1
    
    # add card dict to list of cards
    cards_list.append(card)

In [9]:
# prepare new DataFrame from list of dicts
df = pd.DataFrame(cards_list).set_index('name')
# export to CSV
df.to_csv('table17aug21.csv')